In [1]:
import pandas as pd
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split

In [2]:
combined_stocks_df = pd.read_csv("filtered_stocks_combined.csv")

In [3]:
combined_stocks_df

,Date,index,Open,High,Low,Close,Volume,OpenInt,ticker
0,2011-01-03,45725,41.600,42.293,41.600,42.204,123930383,0,AAPL
1,2011-01-04,45726,42.579,42.579,42.023,42.426,86135637,0,AAPL
2,2011-01-05,45727,42.260,42.817,42.196,42.772,70669988,0,AAPL
3,2011-01-06,45728,42.902,42.932,42.632,42.739,83619699,0,AAPL
4,2011-01-07,45729,42.787,43.073,42.503,43.045,86506108,0,AAPL
...,...,...,...,...,...,...,...,...,...
75495,2016-12-23,14745947,87.556,87.625,87.201,87.383,4428429,0,XOM
75496,2016-12-27,14745948,87.499,87.768,87.257,87.423,5100402,0,XOM
75497,2016-12-28,14745949,87.354,87.768,86.949,86.989,6834213,0,XOM
75498,2016-12-29,14745950,86.797,87.277,86.728,87.036,6938299,0,XOM


In [4]:
# Convert Date to datetime and set as index
combined_stocks_df["Date"] = pd.to_datetime(combined_stocks_df["Date"])
combined_stocks_df.set_index("Date", inplace=True)

# Drop unnecessary columns if any
combined_stocks_df = combined_stocks_df.drop(columns=["index"])  # Optional

# Pivot to multi-level columns: Ticker as level 1, feature as level 2
stocks_df = combined_stocks_df.pivot_table(
    index=combined_stocks_df.index,
    columns="ticker",
    values=[col for col in combined_stocks_df.columns if col != "ticker"]
)

# Sort columns for clarity
stocks_df = stocks_df.sort_index(axis=1, level=0)

# Swap the column MultiIndex levels
stocks_df_leveled = stocks_df.swaplevel(axis=1)

# Sort by ticker (Level 0)
stocks_df_leveled = stocks_df_leveled.sort_index(axis=1, level=0)

# Preview the new structure
stocks_df_leveled.head()


ticker        AAPL                                                  ABT  \
             Close    High     Low    Open OpenInt       Volume   Close   
Date                                                                      
2011-01-03  42.204  42.293  41.600  41.600     0.0  123930383.0  19.358   
2011-01-04  42.426  42.579  42.023  42.579     0.0   86135637.0  19.540   
2011-01-05  42.772  42.817  42.196  42.260     0.0   70669988.0  19.540   
2011-01-06  42.739  42.932  42.632  42.902     0.0   83619699.0  19.500   
2011-01-07  43.045  43.073  42.503  42.787     0.0   86506108.0  19.582   

ticker                              ...     WMT                              \
              High     Low    Open  ...     Low    Open OpenInt      Volume   
Date                                ...                                       
2011-01-03  19.549  19.338  19.549  ...  46.027  46.089     0.0  16789275.0   
2011-01-04  19.561  19.350  19.448  ...  46.139  46.419     0.0  14296931.0   
2011-01-05  19.764  19.460  19.520  ...  46.174  46.539     0.0  16723328.0   
2011-01-06  19.682  19.342  19.634  ...  45.712  46.166     0.0  18335156.0   
2011-01-07  19.615  19.444  19.489  ...  45.698  45.810     0.0   9374462.0   

ticker         XOM                                              
             Close    High     Low    Open OpenInt      Volume  
Date                                                            
2011-01-03  60.341  60.535  59.604  59.668     0.0  28807445.0  
2011-01-04  60.625  60.665  60.235  60.469     0.0  24744869.0  
2011-01-05  60.462  60.600  60.009  60.430     0.0  20448359.0  
2011-01-06  60.853  61.052  60.439  60.625     0.0  27829692.0  
2011-01-07  61.182  61.431  60.777  60.876     0.0  23838996.0  

[5 rows x 300 columns]

In [5]:
# 3. Time-based split into 60% train, 20% val, 20% test
train_dict, val_dict, test_dict = {}, {}, {}

for ticker in stocks_df_leveled.columns.levels[0]:
    stocks_df_leveled.loc[:, (ticker, 'log_return')] = np.log(
    stocks_df_leveled[ticker]['Close'] / stocks_df_leveled[ticker]['Close'].shift(1)
)

    df = stocks_df_leveled[ticker].dropna().sort_index()
    total_len = len(df)
    train_end = int(total_len * 0.6)
    val_end = train_end + int(total_len * 0.2)

    train_dict[ticker] = df.iloc[:train_end]
    val_dict[ticker] = df.iloc[train_end:val_end]
    test_dict[ticker] = df.iloc[val_end:]

train_df = pd.concat(train_dict, names=["Ticker", "Date"])
val_df = pd.concat(val_dict, names=["Ticker", "Date"])
test_df = pd.concat(test_dict, names=["Ticker", "Date"])


In [31]:
train_df

Close    High     Low    Open  OpenInt       Volume  \
Ticker Date                                                               
AAPL   2011-01-04  42.426  42.579  42.023  42.579      0.0   86135637.0   
       2011-01-05  42.772  42.817  42.196  42.260      0.0   70669988.0   
       2011-01-06  42.739  42.932  42.632  42.902      0.0   83619699.0   
       2011-01-07  43.045  43.073  42.503  42.787      0.0   86506108.0   
       2011-01-10  43.855  43.956  43.179  43.393      0.0  124888228.0   
...                   ...     ...     ...     ...      ...          ...   
XOM    2014-08-04  88.145  88.411  86.542  86.921      0.0   13804459.0   
       2014-08-05  86.445  87.625  86.077  87.492      0.0   14847864.0   
       2014-08-06  87.133  87.723  86.445  86.445      0.0   11264688.0   
       2014-08-07  86.507  87.889  85.944  87.828      0.0   11379351.0   
       2014-08-08  87.800  87.847  86.551  86.947      0.0   12730751.0   

                   log_return  
Ticker Date                    
AAPL   2011-01-04    0.005246  
       2011-01-05    0.008122  
       2011-01-06   -0.000772  
       2011-01-07    0.007134  
       2011-01-10    0.018643  
...                       ...  
XOM    2014-08-04    0.013374  
       2014-08-05   -0.019475  
       2014-08-06    0.007927  
       2014-08-07   -0.007210  
       2014-08-08    0.014836  

[45250 rows x 7 columns]